In [20]:
import pydantic
from ga4gh.cat_vrs.core_models import CategoricalVariant, DefiningContextConstraint

json_data=open("cat-vrs-test-set.json", "r").read()
cat_vars=CategoricalVariant.model_validate_json(json_data) # This should be updated to actually intake more than 1 variant
print(cat_vars.model_dump_json())

{"id":"clinvar:662001","type":"CategoricalVariant","label":"NM_004958.4(MTOR):c.5992_5993del (p.Met1998fs)","description":null,"alternativeLabels":null,"extensions":[{"name":"cytogenetic location","value":"1p36.22","description":null},{"name":"clinvar variation type","value":"Deletion","description":null},{"name":"clinvar subclass type","value":"SimpleAllele","description":null},{"name":"hgvs list","value":[{"nucleotideExpression":{"syntax":"hgvs.g","value":"NC_000001.11:g.11128044_11128045del"},"nucleotideType":"genomic, top-level"},{"nucleotideExpression":{"syntax":"hgvs.g","value":"NC_000001.10:g.11188101_11188102del"},"nucleotideType":"genomic"},{"nucleotideExpression":{"syntax":"hgvs.c","value":"NM_004958.4:c.5992_5993del"},"nucleotideType":"coding","maneSelect":true,"proteinExpression":{"syntax":"hgvs.p","value":"NP_004949.3:p.Met1998fs"},"molecularConsequence":{"code":"SO:0001589","system":"http://www.sequenceontology.org/browser/current_release/term/","label":"frameshift_varian

In [22]:
import pysam

from ga4gh.vrs.dataproxy import create_dataproxy
seqrepo_rest_service_url = "https://services.genomicmedlab.org/seqrepo"

from ga4gh.vrs.extras.vcf_annotation import VCFAnnotator, SeqRepoProxyType
input_vcf_path="./test_vcf_input.vcf"
input_vcf_path = pysam.VariantFile(filename=input_vcf_path)
annotator=VCFAnnotator(SeqRepoProxyType.REST, seqrepo_rest_service_url)
additional_info_fields = [annotator.VRS_ALLELE_IDS_FIELD]

In [21]:

def match_vrs_id_to_cat_vrs(vrs_id: str, cat_var: CategoricalVariant):
    constraints = cat_var.constraints
    for constraint in constraints:
        if isinstance(constraint, DefiningContextConstraint) and constraint.definingContext.id == vrs_id:
            print("match")
        elif isinstance(constraint, DefiningContextConstraint) and constraint.definingContext.id != vrs_id:
            print(constraint.definingContext.id + " did not match " + vrs_id)


for record in input_vcf_path:
    vrs_field_data = annotator._get_vrs_data(record, {}, "GRCh38", additional_info_fields, output_vcf="testvcf.vcf", require_validation=False)
    vrs_ids = vrs_field_data[annotator.VRS_ALLELE_IDS_FIELD]
    for vrs_id in vrs_ids:
        if vrs_id == "": continue
        match_vrs_id_to_cat_vrs(vrs_id, cat_vars)
